# Data resampling  

> [https://github.com/BMClab/covid19](https://github.com/BMClab/covid19)  
> [Laboratory of Biomechanics and Motor Control](http://pesquisa.ufabc.edu.br/bmclab/)  
> Federal University of ABC, Brazil

**This Jupyter notebook uses data created by the notebook `preprocessing.ipynb` and we are not sharing this data publicly for ethical reasons. If you want to run this notebook, you will first have to run the 'Strava web scrapper' and the 'Data preprocessing' notebooks we provide.**

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Environment</a></span></li><li><span><a href="#Helping-functions" data-toc-modified-id="Helping-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Helping functions</a></span></li></ul></li><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load dataset</a></span><ul class="toc-item"><li><span><a href="#Checking-for-missing-values" data-toc-modified-id="Checking-for-missing-values-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Checking for missing values</a></span></li><li><span><a href="#Basic-information-about-the-dataset" data-toc-modified-id="Basic-information-about-the-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Basic information about the dataset</a></span></li></ul></li><li><span><a href="#Data-resampling" data-toc-modified-id="Data-resampling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data resampling</a></span><ul class="toc-item"><li><span><a href="#Verify-data" data-toc-modified-id="Verify-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Verify data</a></span></li><li><span><a href="#Export-data" data-toc-modified-id="Export-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Export data</a></span></li><li><span><a href="#Resample-by-year-and-at-different-periods" data-toc-modified-id="Resample-by-year-and-at-different-periods-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Resample by year and at different periods</a></span></li><li><span><a href="#Test-files" data-toc-modified-id="Test-files-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Test files</a></span></li></ul></li></ul></div>

## Setup

In [1]:
import sys, os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
%load_ext watermark  

%watermark
%watermark --iversions

Last updated: 2021-09-14T00:06:41.314629-03:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.27.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-34-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

pandas  : 1.3.3
json    : 2.0.9
autopep8: 1.5.6
sys     : 3.8.10 | packaged by conda-forge | (default, May 11 2021, 07:01:05) 
[GCC 9.3.0]
numpy   : 1.20.3



### Environment

In [2]:
path2 = r'./../data/'

pd.set_option('display.float_format', lambda x: '%.4g' % x)

### Helping functions

In [3]:
def resample(y, cat1='athlete', freq='d', observed=True):
    """Resample numerical columns of `y` and repeat the categorical columns.
    
    y is a dataframe with a datetime index, a categorical column named cat1
    and possibly other categorical columns.
    The resampling period is given by the parameter freq and resample is
    performed at the datetime index and data are grouped by datetime and the
    categorical column given by the parameter cat1.
    Returns the resampled dataframe and the number of elements in each index
    level at each freq period.
    """
    # numerical columns
    cols_num = y.select_dtypes(include='number').columns.to_list()
    # categorical columns
    cols_cat = y.select_dtypes(include='category').columns.to_list()
    if len(cols_cat) > 1:
        y2 = y.drop_duplicates(subset=cat1)[cols_cat
                                           ].sort_values(cat1).reset_index(drop=True)
    cols_cat.remove(cat1)
    # change the dates of the last days to avoid a last week with less than 7 days
    nlastdays = 7 + y.index[-1].dayofyear % 7
    if freq.lower() == '7d' and nlastdays > 7:
        ts = pd.Timestamp(y.index[-1].date() - pd.to_timedelta(nlastdays-7, unit='D'))
        y.index = y.index.where(y.index <= ts, ts)
    t0 = pd.to_datetime({'year':[y.index[0].year], 'month':[1], 'day':[1]})[0]
    grouper = pd.Grouper(axis=0, freq=freq, sort=True, origin=t0)
    # resample only the numerical columns
    y = y.groupby([grouper, cat1], sort=True, observed=observed).sum().reset_index(level=1)
    #y.fillna(0, inplace=True)
    # calculate acumulated runs for numerical columns based on freq period   
    # just correct for differences in length of week, month or year
    if freq.lower() == 'd':
        pass
    elif freq.lower() == '7d':
        y[cols_num] = y[cols_num]
        # correct the divisor if the last week doesn't have 7 days
        y.loc[y.index[-1], cols_num] = y.loc[y.index[-1], cols_num] * (7 / nlastdays)
    elif freq.lower() == 'm':  # faster than using apply if not too many months
        for year in y.index.year.unique().astype(str):
            for month in y.loc[year].index.month.unique().astype(str):
                date = '{}-{}'.format(year, month)
                ndays = pd.Period(date).daysinmonth        
                y.loc[date, cols_num] = y.loc[date, cols_num] * (30 / ndays)
    elif freq.lower() == 'y':
        for year in y.index.year.unique().astype(str):
            date = year
            ndays = 366 if pd.Period(date).is_leap_year else 365   
            y.loc[date, cols_num] = y.loc[date, cols_num] * (365 / ndays)
    else:
        print('No averages are calculated. Only the sum is reported.')
    # number of elements in each index level at each freq period
    nidx0 = y.groupby(level=0, observed=True).size()
    nidx1 = y.groupby(cat1, observed=True).size()
    # add back the categorical columns
    if len(cols_cat):
        y = y.join(y2.set_index(cat1), on=cat1)
    
    return y, nidx0, nidx1

## Load dataset

In [4]:
df = pd.read_parquet(os.path.join(path2, 'run_ww_2019_2020.parquet'))
#df = df[['datetime', 'athlete', 'gender', 'age_group', 'distance', 'duration']]
df['athlete'] = df['athlete'].astype('category')  # bug in parquet
df

,datetime,athlete,gender,age_group,distance,duration,country,major
0,2019-01-01 00:00:00,261,F,18 - 34,4.71,27.25,Canada,CHICAGO 2018
1,2019-01-01 00:00:00,1575,M,35 - 54,6.5,23.63,United States,BOSTON 2019
2,2019-01-01 00:00:00,2010,M,35 - 54,6.64,31.95,United States,NEW YORK 2018
3,2019-01-01 00:00:00,2937,M,35 - 54,42.34,194,Germany,BERLIN 2019
4,2019-01-01 00:00:00,3172,M,18 - 34,6.59,38.58,United States,BOSTON 2019
...,...,...,...,...,...,...,...,...
10703685,2020-12-31 23:54:00,223,M,35 - 54,5.21,30.1,China,CHICAGO 2015
10703686,2020-12-31 23:56:00,20357,F,35 - 54,1.62,8.717,United States,BOSTON 2019
10703687,2020-12-31 23:58:00,16899,F,18 - 34,6.57,49.5,United States,NEW YORK 2017
10703688,2020-12-31 23:59:00,6134,M,35 - 54,5.56,34.05,United States,NEW YORK 2019


### Checking for missing values

In [5]:
n = 0
for col in df:
    null = df[df[col].isnull()]['athlete'].unique().tolist()
    if null:
        print('Athlete: {}, null value in {}'.format(null, col))
        n = 1
if n == 0:
    print('No missing values found.')

Athlete: [15589, 34600, 25997, 32351, 5469, 11733, 4140, 23337, 3191, 8762, 6843, 25509, 35224, 33313, 33673, 4126, 12534, 24620, 35038, 6239, 35664, 36185, 31524, 9504, 16862, 18537, 4893, 8188, 37123, 31754, 26189, 29991, 32797, 3274, 28610, 31809, 5596, 9820, 10298, 9079, 13248, 34145, 31158, 18682, 14830, 2073, 12513, 34235, 29634, 21728, 3190, 16040, 22052, 753, 26483, 24447, 22942, 4497, 10370, 20903, 15822, 34117, 2554, 977, 2543, 23175, 18645, 31482, 4268, 24743, 27611, 36232, 16359, 15246, 24520, 16605, 23350, 21102, 3968, 13693, 23787, 35104, 11248, 8358, 13819, 36624, 12232, 993, 26490, 19086, 15225, 16811, 24084, 4479, 12323, 24914, 8167, 27229, 29213, 34682, 12627, 5904, 26063, 37372, 7355, 29338, 9749, 33766, 33751, 95, 26867, 2842, 15999, 8862, 35515, 21841, 22772, 34530, 28106, 10862, 34331, 26001, 35209, 36002, 35962, 19960, 35671, 3814, 33030, 17160, 23597, 14051, 31744, 5743, 21182, 20933, 4365, 30596, 8116, 17230, 36611, 11139, 26308, 6973, 15759, 2516, 29233, 15873

### Basic information about the dataset

In [6]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10703690 entries, 0 to 10703689
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   datetime   datetime64[ns]
 1   athlete    category      
 2   gender     category      
 3   age_group  category      
 4   distance   float64       
 5   duration   float64       
 6   country    category      
 7   major      category      
dtypes: category(5), datetime64[ns](1), float64(2)
memory usage: 348.5 MB


In [7]:
nday = df['datetime'].dt.date.value_counts().size
print('Number of days:', nday)
nathlete = df['athlete'].unique().size
print('Number of athletes:', nathlete)
nactivity = df.shape[0]
print('Number of running activities:', nactivity)

Number of days: 731
Number of athletes: 36412
Number of running activities: 10703690


## Data resampling

Resample data using a custom function to speedup the process.  
This process consumes at peak about 10 GB of RAM memory. The process could be divided by year, but because we want to generate all possible categories for days and athletes, we first would have to fill each year with all  athletes.

In [8]:
df.set_index('datetime', inplace=True)
df, nathletes, nruns = resample(df, cat1='athlete', freq='d', observed=False)
df.reset_index(inplace=True)

### Verify data

We can see that the Cartesian product of days and athletes was performed and the distance and duration columns were filled with zeros when there was no register for an athlete on a given day:

In [9]:
df

,datetime,athlete,distance,duration,gender,age_group,country,major
0,2019-01-01,0,0,0,F,18 - 34,United States,CHICAGO 2019
1,2019-01-01,1,5.27,30.2,M,35 - 54,Germany,BERLIN 2016
2,2019-01-01,2,0,0,M,35 - 54,United Kingdom,"LONDON 2018,LONDON 2019"
3,2019-01-01,3,10.5,43.95,M,18 - 34,United Kingdom,LONDON 2017
4,2019-01-01,4,9.66,48.65,M,35 - 54,United States,BOSTON 2017
...,...,...,...,...,...,...,...,...
26617167,2020-12-31,37594,20,79,M,18 - 34,United Kingdom,BERLIN 2017
26617168,2020-12-31,37595,0,0,M,18 - 34,United States,"BERLIN 2019,NEW YORK 2015"
26617169,2020-12-31,37596,25.76,108,M,18 - 34,United States,BOSTON 2017
26617170,2020-12-31,37597,0,0,F,18 - 34,United States,BOSTON 2015


In [10]:
nathlete * nday

26617172

In [11]:
nathletes

datetime
2019-01-01    36412
2019-01-02    36412
2019-01-03    36412
2019-01-04    36412
2019-01-05    36412
              ...  
2020-12-27    36412
2020-12-28    36412
2020-12-29    36412
2020-12-30    36412
2020-12-31    36412
Length: 731, dtype: int64

In [12]:
df.drop_duplicates(subset='athlete')[['athlete', 'age_group', 'gender']
                                    ].groupby(['age_group', 'gender']
                                             ).count().unstack(level=0)

athlete              
age_group 18 - 34 35 - 54  55 +
gender                         
F            3864    4629   396
M            8378   16972  2173

In [13]:
df.groupby([df['datetime'].dt.year]).describe()

distance                                     duration              \
             count  mean   std min 25% 50%  75%   max     count  mean   std   
datetime                                                                      
2019     1.329e+07 4.176 7.335   0   0   0 7.52 390.3 1.329e+07 22.93 43.55   
2020     1.333e+07 3.865 6.662   0   0   0 7.07 347.9 1.333e+07 21.39 39.27   

                                 
         min 25% 50%   75%  max  
datetime                         
2019       0   0   0 40.88 2388  
2020       0   0   0 39.93 2300

In [14]:
df.groupby([df['datetime'].dt.year, 'age_group', 'gender']).describe()

distance                                     \
                              count  mean   std min 25% 50%  75%   max   
datetime age_group gender                                                
2019     18 - 34   F       1.41e+06 3.492 6.557   0   0   0 5.83 180.4   
                   M      3.058e+06 4.219 7.452   0   0   0 7.57 221.1   
         35 - 54   F       1.69e+06 3.961  6.97   0   0   0 6.86 217.8   
                   M      6.195e+06 4.343 7.537   0   0   0 8.05 390.3   
         55 +      F      1.445e+05 4.318 7.068   0   0   0 7.71 162.4   
                   M      7.931e+05 4.356 7.321   0   0   0 8.06 238.8   
2020     18 - 34   F      1.414e+06 3.155 5.779   0   0   0 5.31 168.6   
                   M      3.066e+06 3.919 6.818   0   0   0 7.09 201.4   
         35 - 54   F      1.694e+06 3.722 6.306   0   0   0 6.77 167.1   
                   M      6.212e+06 3.999 6.851   0   0   0 7.61 347.9   
         55 +      F      1.449e+05  4.29 6.503   0   0   0 8.05 109.4   
                   M      7.953e+05 4.092 6.689   0   0   0 8.05 266.2   

                           duration                                     
                              count  mean   std min 25% 50%   75%  max  
datetime age_group gender                                               
2019     18 - 34   F       1.41e+06 19.92  39.8   0   0   0 33.53 1920  
                   M      3.058e+06  21.4 40.81   0   0   0 38.37 2153  
         35 - 54   F       1.69e+06 23.99 45.41   0   0   0 41.92 2300  
                   M      6.195e+06 23.58 44.49   0   0   0  42.5 2388  
         55 +      F      1.445e+05 28.75 51.07   0   0   0  49.1 2115  
                   M      7.931e+05 25.87 46.54   0   0   0  46.7 1989  
2020     18 - 34   F      1.414e+06 18.09  35.1   0   0   0 31.88 1439  
                   M      3.066e+06 19.91 36.71   0   0   0 36.87 1917  
         35 - 54   F      1.694e+06 22.77 41.09   0   0   0 42.07 1612  
                   M      6.212e+06 21.88 40.09   0   0   0 41.07 2300  
         55 +      F      1.449e+05 29.66    48   0   0   0  52.7 1297  
                   M      7.953e+05 24.73 42.81   0   0   0 46.75 1857

### Export data

See its [docs](https://arrow.apache.org/docs/python/feather.html) and a comparison on [formats to save Pandas data](https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d).

In [15]:
df.to_parquet(os.path.join(path2, 'run_ww_2019_2020_d.parquet'))

### Resample by year and at different periods

Now that we have a DataFrame with all possible categories for athletes in all days (from the Cartesian product between days and athletes, setting the parameter `observed` to `False` in the `resample` function), we can resample the dataset separately by year and at different periods.

In [16]:
years = ['2019', '2020']
freqs = ['d', '7d', 'm']
df.set_index('datetime', inplace=True)
for year in tqdm(years, desc='Year'):
    for freq in tqdm(freqs, desc='Freq'):
        dfi = resample(df.loc[year], cat1='athlete', freq=freq, observed=True)[0]
        dfi.reset_index(inplace=True)
        if freq == '7d': freq = 'w'
        dfi.to_parquet(os.path.join(path2, 'run_ww_{}_{}.parquet'.format(year, freq)))

Year:   0%|          | 0/2 [00:00<?, ?it/s]

Freq:   0%|          | 0/3 [00:00<?, ?it/s]

Freq:   0%|          | 0/3 [00:00<?, ?it/s]

### Test files

In [17]:
for year in years:
    for freq in freqs:
        if freq == '7d': freq = 'w'
        df = pd.read_parquet(os.path.join(path2, 'run_ww_{}_{}.parquet'.format(year, freq)))
        df['athlete'] = df['athlete'].astype('category')  # bug in parquet
        #df.set_index('datetime', inplace=True)
        print('\nFile: run_ww_{}_{}.parquet'.format(year, freq))
        display(df.drop_duplicates(subset='athlete')[['athlete', 'age_group', 'gender']
                                                    ].groupby(['age_group', 'gender']
                                                             ).count().unstack(level=0))
        display(df.groupby([df['datetime'].dt.year, 'age_group', 'gender']).describe())


File: run_ww_2019_d.parquet


athlete              
age_group 18 - 34 35 - 54  55 +
gender                         
F            3864    4629   396
M            8378   16972  2173

distance                                     \
                              count  mean   std min 25% 50%  75%   max   
datetime age_group gender                                                
2019     18 - 34   F       1.41e+06 3.492 6.557   0   0   0 5.83 180.4   
                   M      3.058e+06 4.219 7.452   0   0   0 7.57 221.1   
         35 - 54   F       1.69e+06 3.961  6.97   0   0   0 6.86 217.8   
                   M      6.195e+06 4.343 7.537   0   0   0 8.05 390.3   
         55 +      F      1.445e+05 4.318 7.068   0   0   0 7.71 162.4   
                   M      7.931e+05 4.356 7.321   0   0   0 8.06 238.8   

                           duration                                     
                              count  mean   std min 25% 50%   75%  max  
datetime age_group gender                                               
2019     18 - 34   F       1.41e+06 19.92  39.8   0   0   0 33.53 1920  
                   M      3.058e+06  21.4 40.81   0   0   0 38.37 2153  
         35 - 54   F       1.69e+06 23.99 45.41   0   0   0 41.92 2300  
                   M      6.195e+06 23.58 44.49   0   0   0  42.5 2388  
         55 +      F      1.445e+05 28.75 51.07   0   0   0  49.1 2115  
                   M      7.931e+05 25.87 46.54   0   0   0  46.7 1989


File: run_ww_2019_w.parquet


athlete              
age_group 18 - 34 35 - 54  55 +
gender                         
F            3864    4629   396
M            8378   16972  2173

...

athlete              
age_group 18 - 34 35 - 54  55 +
gender                         
F            3864    4629   396
M            8378   16972  2173

distance                                          \
                              count  mean   std min   25%   50%   75%   max   
datetime age_group gender                                                     
2020     18 - 34   F      2.009e+05 22.11 27.46   0     0  11.5 35.48 528.5   
                   M      4.357e+05 27.46  32.9   0     0  15.9 43.42 476.8   
         35 - 54   F      2.407e+05 26.07 27.19   0     0 19.68 41.92 579.2   
                   M      8.825e+05 28.02 30.66   0     0 19.39  44.4 624.6   
         55 +      F      2.059e+04 30.06 26.71   0  5.27 26.74 46.84 281.7   
                   M       1.13e+05 28.66 28.02   0 2.876 23.03 45.08 384.4   

                           duration                                         
                              count  mean   std min   25%   50%   75%  max  
datetime age_group gender                                                   
2020     18 - 34   F      2.009e+05 126.8 158.1   0     0  70.9 206.3 6913  
                   M      4.357e+05 139.5 165.8   0     0 84.61 222.9 5421  
         35 - 54   F      2.407e+05 159.5 167.2   0     0   124 253.3 5408  
                   M      8.825e+05 153.2 168.4   0     0 108.8 242.9 5065  
         55 +      F      2.059e+04 207.8 201.4   0 39.57 182.8 309.6 3285  
                   M       1.13e+05 173.3 173.3   0 20.31   141 268.4 3302


File: run_ww_2020_m.parquet


athlete              
age_group 18 - 34 35 - 54  55 +
gender                         
F            3864    4629   396
M            8378   16972  2173

distance                                          \
                              count  mean   std min   25%   50%   75%   max   
datetime age_group gender                                                     
2020     18 - 34   F      4.637e+04 94.71 110.9   0 3.405 55.17 149.2  1600   
                   M      1.005e+05 117.6   132   0 10.43 73.63   181  1377   
         35 - 54   F      5.555e+04 111.7 108.5   0 17.16 88.22 173.8  1962   
                   M      2.037e+05   120 122.1   0 17.87 87.62 184.7  1560   
         55 +      F           4752 128.7 106.2   0 35.59 117.4 197.5 935.5   
                   M      2.608e+04 122.8 110.8   0 29.97 101.9 186.9  1020   

                           duration                                    \
                              count  mean   std min   25%   50%   75%   
datetime age_group gender                                               
2020     18 - 34   F      4.637e+04 543.1 624.9   0  23.5 337.4 865.8   
                   M      1.005e+05 597.5 650.8   0    60 396.2 930.6   
         35 - 54   F      5.555e+04 683.3 651.2   0 117.1 562.7  1057   
                   M      2.037e+05 656.6 654.7   0 107.6 494.4  1011   
         55 +      F           4752   890 793.4   0 261.3 806.4  1295   
                   M      2.608e+04 742.2 673.7   0 197.6 625.5  1112   

                                     
                                max  
datetime age_group gender            
2020     18 - 34   F      2.215e+04  
                   M           9243  
         35 - 54   F      1.699e+04  
                   M      1.071e+04  
         55 +      F           9606  
                   M           8395